#  Standards - Calculate minimum standoff distance for roof mounted systems

This example demonstrates the calculation of a minimum standoff distance necessary for roof-mounted PV modules to ensure that the $T_{98}$ operational temperature remains under 70°C, in which case the more rigorous thermal stability testing requirements of IEC TS 63126 would not needed to be considered. We use data from [Fuentes, 1987] to model the approximate exponential decay in temperature, $T(X)$, with increasing standoff distance, $X$, as,

$$ X = -X_0 \ln\left(1-\frac{T_0-T}{\Delta T}\right)$$

where $T_0$ is the temperature for $X=0$ (insulated back) and $\Delta T$ is the temperature difference between an insulated back ($X=0$) and open rack mounting configuration ($X=\infty)$.

The following figure showcases this calulation for the entire United States. We used pvlib and data from the National Solar Radiation Database (NSRDB) to calculate the module temperatures for different mounting configuration and applied our model to obtain the standoff distance for roof-mounted PV systems.

# Single location example

In [5]:
import pvdeg

## NSRDB - HSDS on Eagle

In [ ]:
#Get weather data
weather_db = 'NSRDB'
weather_id = (39.741931, -105.169891)
#weather_id = 1933572  
weather_arg = {'satellite' : 'GOES', 
               'names' : 2021, 
               'NREL_HPC' : True,
               'attributes' : ['air_temperature', 'wind_speed', 'dhi', 'ghi', 'dni','relative_humidity']}

weather_df, meta = pvdeg.weather.load(weather_db, weather_id, **weather_arg)

#Perform calculation
res = pvdeg.standards.calc_standoff(weather_df, meta, tilt=None, azimuth=180, sky_model='isotropic', temp_model='sapm', 
                    module_type='glass_polymer', level=1, x_0=6.1)
print(res)

## NSRDB - API

In [3]:
#Get weather data
API_KEY = 'your_api_key_here' 
#The example API key here is for demonstation and is rate-limited per IP. 
#To get your own API key, visit https://developer.nrel.gov/signup/

weather_db = 'PSM3'
weather_id = (39.741931, -105.169891)
weather_arg = {'api_key': API_KEY,
               'email': 'user@mail.com',
               'names' : 'tmy', 
               'map_variables': True}

weather_df, meta = pvdeg.weather.load(weather_db, weather_id, **weather_arg)

#Perform calculation
res = pvdeg.standards.calc_standoff(weather_df, meta, tilt=None, azimuth=180, sky_model='isotropic', temp_model='sapm', 
                    module_type='glass_polymer', level=1, x_0=6.1)
print(res)

{'x': 2.0063800910065974, 'T98_0': 77.85091991839371, 'T98_inf': 49.84149463751572}


## PVGIS - API

In [13]:
weather_db = 'PVGIS'
weather_id = (39.741931, -105.169891)
#weather_id = (48.667, 15.885)
weather_arg = {'map_variables': True}

#TMY
weather_df, meta = pvdeg.weather.load(weather_db, weather_id, **weather_arg)

#Perform calculation
# res = pvdeg.standards.calc_standoff(weather_df, meta, tilt=None, azimuth=180, sky_model='isotropic', temp_model='sapm', 
#                     module_type='glass_polymer', level=1, x_0=6.1)
# print(res)